In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
# !pip install catboost

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (8,005 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

from sklearn.ensemble import RandomForestClassifier

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/와인/train.csv')
test = pd.read_csv('/content/drive/MyDrive/와인/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/와인/sample_submission.csv')

In [ ]:
train = train.append(train[train['quality'].isin([4,7,8])])
train = pd.concat([train,pd.get_dummies(train['type'])], axis=1)
train

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type,red,white
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white,0,1
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red,1,0
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white,0,1
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white,0,1
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5481,5481,7,5.4,0.835,0.08,1.2,0.046,13.0,93.0,0.99240,3.57,0.85,13.0,red,1,0
5482,5482,4,6.2,0.430,0.49,6.4,0.045,12.0,115.0,0.99630,3.27,0.57,9.0,white,0,1
5485,5485,4,6.3,0.280,0.22,9.5,0.040,30.0,111.0,0.99338,3.05,0.31,10.8,white,0,1
5491,5491,7,6.9,0.340,0.30,4.7,0.029,34.0,148.0,0.99165,3.36,0.49,12.3,white,0,1


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train[train.columns[2:-3]] = scaler.fit_transform(train[train.columns[2:-3]])

######################
test = pd.concat([test,pd.get_dummies(test['type'])], axis=1)
test[test.columns[1:-3]] = scaler.transform(test[test.columns[1:-3]])
# test

In [ ]:
features = train.columns[2:].drop('type')
features

X = train[features]
y = train['quality']

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.1, shuffle=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:
models = []

rfc = RandomForestClassifier( ) 
models.append(rfc)

gbc = LGBMClassifier(  )
models.append(gbc)

etc = ExtraTreesClassifier()
models.append(etc)

In [ ]:
from sklearn.model_selection import StratifiedKFold

best_models={}
params = [{'n_estimators': [2700],
          'min_samples_leaf': [1],},

          {'metric': ['multi_logloss'],
    'n_estimators': [2800],
    'objective': ['multiclass'],
    # 'random_state': [42],
    'num_leaves': [31],
    'num_class': [6],
    'max_depth': [10],
    'subsample':[0.8],
    'min_child_samples':[2]},
    
    {'bootstrap':[True],
    'criterion':['entropy'], 
    'n_estimators':[3000]}]

cv = StratifiedKFold(n_splits=5, shuffle = True, random_state=42)

for i, model in enumerate(models):
  model = GridSearchCV(model, param_grid=params[i], cv=cv, return_train_score=True, verbose=2)
  
  model.fit(X,y)
  best_models[i] = model.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..............min_samples_leaf=1, n_estimators=2700; total time=  30.4s
[CV] END ..............min_samples_leaf=1, n_estimators=2700; total time=  30.1s
[CV] END ..............min_samples_leaf=1, n_estimators=2700; total time=  28.9s
[CV] END ..............min_samples_leaf=1, n_estimators=2700; total time=  28.8s
[CV] END ..............min_samples_leaf=1, n_estimators=2700; total time=  28.5s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END max_depth=10, metric=multi_logloss, min_child_samples=2, n_estimators=2800, num_class=6, num_leaves=31, objective=multiclass, subsample=0.8; total time=  16.9s
[CV] END max_depth=10, metric=multi_logloss, min_child_samples=2, n_estimators=2800, num_class=6, num_leaves=31, objective=multiclass, subsample=0.8; total time=  16.7s
[CV] END max_depth=10, metric=multi_logloss, min_child_samples=2, n_estimators=2800, num_class=6, num_leaves=31, objective=multiclass, subs

In [ ]:
pred0 = best_models[0].predict(test[features])
pred1 = best_models[1].predict(test[features])
pred2 = best_models[2].predict(test[features])
pred = pd.DataFrame({'pred0':pred0, 'pred1':pred1, 'pred2':pred2})
# pred
pred['pred'] = pred.mode(axis=1)[0].astype(int)
submission['quality'] = pred['pred']
submission

,pred0,pred1,pred2,pred
0,5,5,6,5
1,6,6,6,6
2,6,6,6,6
3,5,5,5,5
4,6,6,6,6
...,...,...,...,...
995,6,6,6,6
996,6,5,6,6
997,5,5,5,5
998,6,6,6,6


In [ ]:
submission.to_csv('submissionFF.csv', index=False)
